In [ ]:
import os
from pathlib import Path

import pandas as pd

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"


current_dir = Path(os.getcwd())

PROJECT = current_dir.parent.resolve()
SRC = PROJECT.parent.resolve()
ROOT = PROJECT.joinpath("..", "..").resolve()

BLD = ROOT.joinpath("bld").resolve()

# Folders inside idos_ppp
DATA = PROJECT.joinpath("data").resolve()
DATA_MGT = PROJECT.joinpath("data_management").resolve()
ANALYSIS = PROJECT.joinpath("analysis").resolve()
FINAL = PROJECT.joinpath("final").resolve()

DOCUMENTS = ROOT.joinpath("documents").resolve()

print(ROOT)

In [ ]:
from idos_ppp.config import BLD

bld_file = BLD
# Construct the path to the "clean_data.csv" file
clean_data_path = bld_file.joinpath("data", "clean_data.csv")

# Open the "clean_data.csv" file using pandas
try:
    clean_data = pd.read_csv(clean_data_path)
    print("Data loaded successfully.")
    # Display the first few rows of the dataframe
    print(clean_data.head())
except FileNotFoundError:
    print(f"Error: The file {clean_data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
clean_data